# Extract InceptionResNet bottleneck features
From raw image, get 1536 features of the penultimate InceptionResNet layer.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import _pickle as pickle
from os import listdir
from os.path import join, isfile

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras import backend as K
K.set_session(session)

from keras.preprocessing import image
from keras.utils import to_categorical, plot_model

from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score


Using TensorFlow backend.


In [2]:
def read_img(img_id, train_or_test, size=None):
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [3]:
NUM_CLASSES = 120
SEED = 1993
np.random.seed(seed=SEED)
data_dir = '../data'

labels = pd.read_csv(join(data_dir, 'labels.csv'))
print('Number of all train images: {}'.format(len(labels)))
print("Train data has {} classes.".format(len(labels.groupby('breed').count())))
assert len(labels.groupby('breed').count()) == NUM_CLASSES, 'Number of classes in training set is not 120!'

sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print('Number of all test images: {}'.format(len(sample_submission)))

# Split to train and validation sets
l_val = labels.groupby('breed').apply(pd.DataFrame.sample, frac=0.2).reset_index(drop=True)
l_tr = labels.loc[~labels['id'].isin(l_val['id'])]
l_tr_index = {label:i for i,label in enumerate(np.unique(l_tr.breed))}
l_tr_temp = [l_tr_index[label] for label in l_tr.breed]
l_val_temp = [l_tr_index[label] for label in l_val.breed]
y_tr = to_categorical(l_tr_temp ,num_classes=120)
y_val = to_categorical(l_val_temp ,num_classes=120)
print('y_tr shape: {}'.format(y_tr.shape))
print('y_val shape: {}'.format(y_val.shape))

Number of all train images: 10222
Train data has 120 classes.
Number of all test images: 10357
y_tr shape: (8185, 120)
y_val shape: (2037, 120)


In [6]:
# Get bottelneck features
INPUT_SIZE = 299
POOLING = 'avg'
filename = data_dir + '//train//xs_bf_irn'

if isfile(filename):
    # Load from file
    print('Loading from {}'.format(filename))
    with open(filename, 'rb') as fp:
        xs_bf = pickle.load(fp)
    print('xs_bf shape: {} size: {:,}'.format(xs_bf.shape, xs_bf.size))
else:
    # Preprocess images
    print('Reading from raw images')
    xs = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
    for i, img_id in enumerate(labels['id']):
        img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
        x = preprocess_input(np.expand_dims(img.copy(), axis=0))
        xs[i] = x
    print('All train Images shape: {} size: {:,}'.format(xs.shape, xs.size))
    
    # Predict
    preprocess_bottleneck = InceptionResNetV2(weights='imagenet', include_top=False, pooling=POOLING)
    xs_bf_raw = preprocess_bottleneck.predict(xs, batch_size=6, verbose=1)
    print('All train bottleneck features shape: {} size: {:,}'.format(xs_bf_raw.shape, xs_bf_raw.size))
    xs_bf = pd.concat([labels.id, pd.DataFrame(xs_bf_raw)], axis=1)
    
    # Save into file
    print('Dumping into {}'.format(filename))
    with open(filename, 'wb') as fp:
        pickle.dump(xs_bf, fp)

# Split to train/val sets
x_tr = np.zeros((len(l_tr), xs_bf.shape[1]-1), dtype='float32')
for i, img_id in enumerate(l_tr['id']):
    x_tr[i] = xs_bf[xs_bf.id == img_id].values[0][1:]
print('Train bottleneck features shape: {} size: {:,}'.format(x_tr.shape, x_tr.size))

x_val = np.zeros((len(l_val), xs_bf.shape[1]-1), dtype='float32')
for i, img_id in enumerate(l_val['id']):
    x_val[i] = xs_bf[xs_bf.id == img_id].values[0][1:]
print('Validation bottleneck features shape: {} size: {:,}'.format(x_val.shape, x_val.size))

xs = None
xs_bf = None
xs_bf_raw = None

Reading from raw images
All train Images shape: (10222, 299, 299, 3) size: 2,741,571,066
10222/10222 [==============================] - 1786s 175ms/step
All train bottleneck features shape: (10222, 1536) size: 15,700,992
Dumping into ../data//train//xs_bf_irs
Train bottleneck features shape: (8185, 1536) size: 12,572,160
Validation bottleneck features shape: (2037, 1536) size: 3,128,832


In [7]:
# Logistic regression on bottleneck features
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(x_tr, (y_tr * range(NUM_CLASSES)).sum(axis=1))
valid_probs = logreg.predict_proba(x_val)
valid_preds = logreg.predict(x_val)
print('Validation LogLoss {}'.format(log_loss(y_val, valid_probs)))
print('Validation Accuracy {}'.format(accuracy_score((y_val * range(NUM_CLASSES)).sum(axis=1), valid_preds)))

Validation LogLoss 0.31869074149026316
Validation Accuracy 0.9072164948453608


In [4]:
# Get test bottelneck features
INPUT_SIZE = 299
POOLING = 'avg'
filename = data_dir + '//test//xs_bf_irn'

if isfile(filename):
    print('File {} already exists!'.format(filename))
else:
    # Preprocess images
    print('Reading from raw images')
    xs = np.zeros((len(sample_submission.id), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
    for i, img_id in enumerate(sample_submission['id']):
        img = read_img(img_id, 'test', (INPUT_SIZE, INPUT_SIZE))
        x = preprocess_input(np.expand_dims(img.copy(), axis=0))
        xs[i] = x
    print('All test Images shape: {} size: {:,}'.format(xs.shape, xs.size))
    
    # Predict
    preprocess_bottleneck = InceptionResNetV2(weights='imagenet', include_top=False, pooling=POOLING)
    xs_bf_raw = preprocess_bottleneck.predict(xs, batch_size=6, verbose=1)
    print('All test bottleneck features shape: {} size: {:,}'.format(xs_bf_raw.shape, xs_bf_raw.size))
    xs_bf = pd.concat([sample_submission.id, pd.DataFrame(xs_bf_raw)], axis=1)
    
    # Save into file
    print('Dumping into {}'.format(filename))
    with open(filename, 'wb') as fp:
        pickle.dump(xs_bf, fp)

xs = None
xs_bf = None
xs_bf_raw = None

Reading from raw images
All test Images shape: (10357, 299, 299, 3) size: 2,777,778,471
10357/10357 [==============================] - 1791s 173ms/step
All test bottleneck features shape: (10357, 1536) size: 15,908,352
Dumping into ../data//test//xs_bf_irs
